In [ ]:
pip install -r requirements.txt

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("en.openfoodfacts.org.products.csv",
              sep="\t", encoding="utf-8", nrows=10000, low_memory=False)

In [3]:
df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,abbreviated_product_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,nutrinet-sante,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1656488106,2022-06-29T07:35:06Z,stephane,L.casei,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0011110072917,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489066521,2017-03-09T13:35:21Z,1626477800,2021-07-16T23:23:20Z,kiliweb,Banana walnut muffins,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,0011110072948,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489067566,2017-03-09T13:52:46Z,1631035185,2021-09-07T17:19:45Z,benbenben,Blueberry muffins,NaN,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,0011110072962,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489068347,2017-03-09T14:05:47Z,1587581916,2020-04-22T18:58:36Z,org-database-usda,"Baked fresh goodness, honey raisin bran muffins",NaN,...,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,0011110073020,http://world-en.openfoodfacts.org/product/0011...,kiliweb,1546804865,2019-01-06T20:01:05Z,1585414154,2020-03-28T16:49:14Z,charlesnepote,Pecan roll coffeecake,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.Filtrer le dataframe

In [15]:
from t0_datafilter import datas_filter

In [9]:
filter = datas_filter()

filter.downcast(df, show_saved_memory=True)

# filter exemples
print("Mise en place des filtres :")
print("Colones numériques : ")
print(list(filter.filter(df, type="number", nan_percent=34).columns))
print("Colones ordinales : ")
#print(list(filter.filter(df, type="ordinal", ordinal_features_names=["i"]).columns))
print("Colones non-ordinales (max 2): ")
print(list(filter.filter(df, type="non-ordinal", category_count=2).columns))
print("Colones catégorielles : ")
print(list(filter.filter(df, type="categorical").columns))

Taille mémoire des données avant filtrage : 41823688 octets
Taille mémoire des données après filtrage : 41823688 octets
Taille mémoire des données sauvegardée : 0 octets
Mise en place des filtres :
Colones numériques : 
['created_t', 'last_modified_t']
Colones ordinales : 
Colones non-ordinales (max 2): 
[]
Colones catégorielles : 
['code', 'url', 'creator', 'created_datetime', 'last_modified_datetime', 'states', 'states_tags', 'states_en']


## 1.0.Nettoyer le jeu de données des valeurs problématiques

In [17]:
from t1_0_cleandata import drop_columns_with_missing_values

In [23]:
drop_columns_with_missing_values(df, threshold=0.95)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,quantity,...,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,nutrinet-sante,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,3.000000,1.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1656488106,2022-06-29T07:35:06Z,stephane,L.casei,NaN,...,NaN,2.699219,0.099976,0.040009,NaN,NaN,NaN,NaN,0.0,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,480 g,...,NaN,10.000000,0.090027,0.036011,NaN,NaN,NaN,NaN,NaN,-5.0
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,7.800781,1.400391,0.560059,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0011110072917,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489066521,2017-03-09T13:35:21Z,1626477800,2021-07-16T23:23:20Z,kiliweb,Banana walnut muffins,NaN,...,0.899902,6.605469,0.895996,0.358398,0.000057,0.0000,0.018997,0.00255,0.0,NaN
9996,0011110072948,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489067566,2017-03-09T13:52:46Z,1631035185,2021-09-07T17:19:45Z,benbenben,Blueberry muffins,15 oz,...,0.899902,5.660156,0.801758,0.320801,0.000028,0.0011,0.018997,0.00255,0.0,18.0
9997,0011110072962,http://world-en.openfoodfacts.org/product/0011...,usda-ndb-import,1489068347,2017-03-09T14:05:47Z,1587581916,2020-04-22T18:58:36Z,org-database-usda,"Baked fresh goodness, honey raisin bran muffins",NaN,...,2.800781,4.718750,0.707520,0.282959,0.000028,0.0000,0.037994,0.00170,0.0,15.0
9998,0011110073020,http://world-en.openfoodfacts.org/product/0011...,kiliweb,1546804865,2019-01-06T20:01:05Z,1585414154,2020-03-28T16:49:14Z,charlesnepote,Pecan roll coffeecake,14 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
